<a href="https://colab.research.google.com/github/aniketinamdar/Twitter-Sentiment-Analysis-using-Support-Vector-Machine-/blob/main/BidirectionalRNNipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing necessary libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.layers import Embedding
from sklearn.model_selection import train_test_split
from nltk.tokenize.treebank import TreebankWordDetokenizer
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

##Reading the csv file

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/Twitter_Data.csv',nrows=10000)

In [ ]:
df.head

<bound method NDFrame.head of                                              clean_text  category
0     when modi promised “minimum government maximum...        -1
1     talk all the nonsense and continue all the dra...         0
2     what did just say vote for modi  welcome bjp t...         1
3     asking his supporters prefix chowkidar their n...         1
4     answer who among these the most powerful world...         1
...                                                 ...       ...
9995  modi made 1000 promises manifesto after electi...         0
9996                  jds leaders also saying modi modi         0
9997  woh sirf modi gaali raha tha and changed his m...         1
9998  you must say what you witnessed since 2014 you...         1
9999  knows once modi comes again his entire family ...        -1

[10000 rows x 2 columns]>

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   clean_text  9999 non-null   object
 1   category    10000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


In [ ]:
df.clean_text = df.clean_text.fillna('')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   clean_text  10000 non-null  object
 1   category    10000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


In [ ]:
df.size

20000

In [ ]:
df.shape

(10000, 2)

##Data Preprocessing

In [ ]:
# df['clean_text'] = df['clean_text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','')
df['clean_text'] = df['clean_text'].astype(str).replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','')

In [ ]:
df['clean_text'] = df['clean_text'].astype(str).replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','')

In [ ]:
df['clean_text'] = df['clean_text'].astype(str).replace(r"[\"\'\|\?\=\.\@\#\*\,]", '')

In [ ]:
df['clean_text'] = df['clean_text'].astype(str).replace("\'", '')

In [ ]:
df.head

<bound method NDFrame.head of                                              clean_text  category
0     when modi promised “minimum government maximum...        -1
1     talk all the nonsense and continue all the dra...         0
2     what did just say vote for modi  welcome bjp t...         1
3     asking his supporters prefix chowkidar their n...         1
4     answer who among these the most powerful world...         1
...                                                 ...       ...
9995  modi made 1000 promises manifesto after electi...         0
9996                  jds leaders also saying modi modi         0
9997  woh sirf modi gaali raha tha and changed his m...         1
9998  you must say what you witnessed since 2014 you...         1
9999  knows once modi comes again his entire family ...        -1

[10000 rows x 2 columns]>

In [ ]:
import re

In [ ]:
new_str = df['clean_text']
new_str = re.sub('"', '', example_str)

NameError: ignored

In [ ]:
df['clean_text']

0       when modi promised “minimum government maximum...
1       talk all the nonsense and continue all the dra...
2       what did just say vote for modi  welcome bjp t...
3       asking his supporters prefix chowkidar their n...
4       answer who among these the most powerful world...
                              ...                        
9995    modi made 1000 promises manifesto after electi...
9996                    jds leaders also saying modi modi
9997    woh sirf modi gaali raha tha and changed his m...
9998    you must say what you witnessed since 2014 you...
9999    knows once modi comes again his entire family ...
Name: clean_text, Length: 10000, dtype: object

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   clean_text  10000 non-null  object
 1   category    10000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True))

In [ ]:
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

In [ ]:
sent_to_words(df['clean_text'])

<generator object sent_to_words at 0x7f3aff0fb430>

In [ ]:
df['clean_text']=detokenize(df['clean_text'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   clean_text  10000 non-null  object
 1   category    10000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


##Test Train split

In [ ]:
X_train, X_test = train_test_split(df['clean_text'], test_size=0.2, random_state=42, shuffle=True)

In [ ]:
Y_train, Y_test = train_test_split(df['category'], test_size=0.2, random_state=42, shuffle=True)

##Converting sentences to word embeddings

In [ ]:
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['clean_text'])
sequences = tokenizer.texts_to_sequences(df['clean_text'])
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

KeyboardInterrupt: ignored

##Embedding Layer

In [ ]:
embedding_layer = Embedding(1000, 64)

##Bidirectional Recurrent Neural Network

In [ ]:
model = Sequential()

In [ ]:
model.add(layers.Embedding(max_words, 40, input_length=max_len))

In [ ]:
model.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))

In [ ]:
model.add(layers.Dense(3,activation='softmax'))

In [ ]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint("best_model2.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)

In [ ]:
X_train.shape

(800,)

In [ ]:
X_test.shape

(200,)

In [ ]:
Y_train.shape

(800,)

In [ ]:
Y_test.shape

(200,)

In [ ]:
history = model.fit(X_train, Y_train, epochs=100,validation_data=(X_test, Y_test),callbacks=[checkpoint])

##Checking the results of the model

In [ ]:
sentiment = ['Neutral','Negative','Positive']
sequence = tokenizer.texts_to_sequences(['this data science article is the best ever'])
test = pad_sequences(sequence, maxlen=max_len)
sentiment[np.around(model.predict(test), decimals=0).argmax(axis=1)[0]]